# To Explore and Process the data



In [5]:
import pandas as pd

In [6]:
ls

 Le volume dans le lecteur C s'appelle Windows
 Le num‚ro de s‚rie du volume est 2AB3-9FC9

 R‚pertoire de C:\Users\Lucas\Documents\GitHub\ML_project2\Data_exploration

23/11/2017  17:17    <DIR>          .
23/11/2017  17:17    <DIR>          ..
23/11/2017  17:13    <DIR>          .ipynb_checkpoints
23/11/2017  17:13               232 build_vocab.sh
23/11/2017  17:13               236 cut_vocab.sh
23/11/2017  17:15           160ÿ231 vocab_cut_neg.txt
23/11/2017  17:15            91ÿ762 vocab_cut_pos.txt
23/11/2017  17:13            54ÿ596 Vocal handling.ipynb
               5 fichier(s)          307ÿ057 octets
               3 R‚p(s)  52ÿ253ÿ077ÿ504 octets libres


In [7]:
path_pos = "./vocab_cut_pos.txt"
path_neg = "./vocab_cut_neg.txt"

In [8]:
pos = pd.read_table(filepath_or_buffer = path_pos, header=None, names=["word"])
pos["occurence"] = pos["word"].map(lambda x:  int(x.split()[0]))
pos["word"] = pos["word"].map(lambda x:  x.split()[1])

neg = pd.read_table(filepath_or_buffer = path_neg, header=None, names=["word"])
neg["occurence"] = neg["word"].map(lambda x:  int(x.split()[0]))
neg["word"] = neg["word"].map(lambda x:  x.split()[1])

In [9]:
pos.set_index?

In [10]:
neg.head()

,word,occurence
0,<user>,46321
1,(,43736
2,i,43412
3,the,34725
4,<url>,34310


In [11]:
pos.head()

,word,occurence
0,<user>,81649
1,!,52891
2,i,37253
3,you,31560
4,.,30368


In [12]:
pd.merge?

In [13]:
# We merge the two dataframe in order to better handle them
#
merged = pd.merge(left=neg, right=pos, left_on = "word", right_on = "word", suffixes=('_neg', '_pos'),  how="outer")
merged = merged.fillna(0)
merged["ratio"] = abs((merged["occurence_neg"]-merged["occurence_pos"]))
merged = merged[merged["ratio"]>=5]
merged["somme"] = merged["occurence_neg"]+merged["occurence_pos"]
merged["ratio"] = 2* abs(0.5 - merged["occurence_pos"]/(merged["occurence_pos"]+merged["occurence_neg"]))
merged.sort_values(by = ["ratio","somme"], ascending=[False, False])

,word,occurence_neg,occurence_pos,ratio,somme
55,paperback,3509.0,0.0,1.000000,3509.0
150,hardcover,1339.0,0.0,1.000000,1339.0
249,ounce,761.0,0.0,1.000000,761.0
293,replacement,625.0,0.0,1.000000,625.0
359,cable,505.0,0.0,1.000000,505.0
373,ram,489.0,0.0,1.000000,489.0
419,steel,439.0,0.0,1.000000,439.0
426,a-tech,432.0,0.0,1.000000,432.0
452,electronics,402.0,0.0,1.000000,402.0
476,ddr,377.0,0.0,1.000000,377.0


In [14]:
merged.sort_values(by = ["ratio","somme"], ascending=[False, False])

,word,occurence_neg,occurence_pos,ratio,somme
55,paperback,3509.0,0.0,1.000000,3509.0
150,hardcover,1339.0,0.0,1.000000,1339.0
249,ounce,761.0,0.0,1.000000,761.0
293,replacement,625.0,0.0,1.000000,625.0
359,cable,505.0,0.0,1.000000,505.0
373,ram,489.0,0.0,1.000000,489.0
419,steel,439.0,0.0,1.000000,439.0
426,a-tech,432.0,0.0,1.000000,432.0
452,electronics,402.0,0.0,1.000000,402.0
476,ddr,377.0,0.0,1.000000,377.0


In [15]:
merged[merged["word"] == "happy"]

,word,occurence_neg,occurence_pos,ratio,somme
290,happy,630.0,2462.0,0.592497,3092.0


In [16]:
merged[merged["word"] == "sad"]

,word,occurence_neg,occurence_pos,ratio,somme
111,sad,1969.0,138.0,0.869008,2107.0


In [19]:
merged.to_csv(path_or_buf="relevant_vocab.txt", sep=' ')